In [ ]:
import os
import cvut
import mmcv
import torch
import numpy as np
from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from mmcv.runner import load_checkpoint
from sklearn.metrics import plot_confusion_matrix

from mmdet.models import build_detector
from mmdet.datasets import build_dataset

In [ ]:
CFG = "../configs/sscod/voc/exp2_arccon.py"
CKPT = "/checkpoints/sscod/voc/exp2_arccon/c8d80951a6df089d_e_12.pth"

cfg = mmcv.Config.fromfile(CFG)
dataset = build_dataset(cfg.data.test)
model = build_detector(cfg.model, train_cfg=None, test_cfg=cfg.test_cfg)
load_checkpoint(model, CKPT, map_location='cpu');

MEAN = np.array(cfg.img_norm_cfg['mean'])
STD = np.array(cfg.img_norm_cfg['std'])

In [ ]:
# Visualize data

idx = 4
sample = dataset[idx]
img = sample['img'][0]

image = cvut.impt2np(img.unsqueeze(0))
image = cvut.imdenormalize(image, MEAN, STD, scale_255=False)

plt.figure(figsize=(10,10))
plt.imshow(image)
plt.show()

In [ ]:
# Inference

data = dict(
    img=[sample['img'][0].unsqueeze(0)],
    img_meta=[[sample['img_meta'][0].data]],
)
outputs = model(return_loss=False, rescale=False, **data)
bboxes, labels, embeddings = outputs['boxes'], outputs['classes'], outputs['embed']

# Threshold
det_thr = 0.3
print("Number of bboxes before thresholding:", len(bboxes))
keep_indicators = bboxes[:,-1] > det_thr
bboxes = bboxes[keep_indicators]
labels = labels[keep_indicators]
embeddings = embeddings[keep_indicators]
print("Number of bboxes after thresholding:", len(bboxes))

# Visualize
labels = np.arange(len(bboxes))
classnames = ["cls"+str(label) for label in labels]
drawn_image = cvut.draw_bboxes(image, bboxes[:,:-1], labels=labels, classnames=classnames, color=None)
plt.figure(figsize=(20,20))
plt.imshow(drawn_image)
plt.show()

In [ ]:
# Embed

def compute_similarity(embeddings1, embeddings2):
    scores = np.matmul(embeddings1, embeddings2.T)
    return scores

def compute_self_similarity(embeddings):
    return compute_similarity(embeddings, embeddings)

scores = compute_self_similarity(embeddings)
plt.figure(figsize=(20,20))
res = plt.imshow(scores, cmap=plt.cm.jet)
plt.colorbar()
plt.xticks(range(len(bboxes)))
plt.yticks(range(len(bboxes)))
plt.show()

In [ ]:
# Draw only serveral objects

objects = np.array([0, 1, 3, 4, 5, 7, 9, 10, 11])

drawn_image = cvut.draw_bboxes(image, bboxes[:,:-1][objects], labels=labels[objects], classnames=classnames, color=None)
plt.figure(figsize=(20,20))
plt.imshow(drawn_image)
plt.show()